In [2]:
import os
import numpy as np
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from PIL import Image, ImageChops, ImageEnhance
import cv2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from sklearn.model_selection import train_test_split
from scipy import stats
from keras.models import Sequential
from keras.layers import BatchNormalization
from tensorflow.keras.applications import Xception

In [24]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp.jpg'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    ela_image = ImageChops.difference(image, temp_image)
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    return ela_image

In [7]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', activation = 'relu', input_shape = (224, 224, 3)))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', activation = 'relu',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model

In [14]:
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0

In [15]:
image_size = (224, 224)
X = []
y = []

In [16]:
# Define source paths
authentic = 'data/casia/au/'
tampered = 'data/casia/tp/'

In [17]:
for root, dirs, files in os.walk(authentic):
  for filename in files:
    file_path = os.path.join(root, filename)
    if filename.lower().endswith(('jpg', 'png')):
      X.append(prepare_image(file_path))
      y.append(1)
    if len(X)==3000:
        break

# random.shuffle(X)

print(len(X), len(y))

37 37


In [18]:
count = 0
for root, dirs, files in os.walk(tampered):
    for filename in files:
        file_path = os.path.join(root, filename)
        if filename.lower().endswith(('jpg', 'png', 'tif', 'tiff')):
            X.append(prepare_image(file_path))
            y.append(0)
        if len(X)==4500:
            break

In [19]:
X = np.array(X)
y = to_categorical(y, 2)
X = X.reshape(-1, 224, 224, 3)

In [20]:
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.2, random_state=5)
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

58 58
15 15


In [21]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5,mode='max', restore_best_weights=True)
early_stopping

In [20]:
model = build_model()
epochs = 30
batch_size = 32
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

2024-12-09 09:26:45.510815: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [21]:
hist = model.fit(
    X_train,
    Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, Y_val),
    callbacks = [early_stopping])
hist

Epoch 1/30
3/3 [==============================] - 2s 472ms/step - loss: 14.5824 - accuracy: 0.6705 - val_loss: 0.3530 - val_accuracy: 0.8182
Epoch 2/30
3/3 [==============================] - 1s 364ms/step - loss: 4.6532 - accuracy: 0.8523 - val_loss: 0.2492 - val_accuracy: 0.9091
Epoch 3/30
3/3 [==============================] - 1s 315ms/step - loss: 5.0055 - accuracy: 0.9545 - val_loss: 0.3948 - val_accuracy: 0.8182
Epoch 4/30
3/3 [==============================] - 1s 320ms/step - loss: 1.8395 - accuracy: 0.9545 - val_loss: 0.9243 - val_accuracy: 0.8182
Epoch 5/30
3/3 [==============================] - 1s 323ms/step - loss: 1.0185 - accuracy: 0.9773 - val_loss: 1.2326 - val_accuracy: 0.8182
Epoch 6/30
3/3 [==============================] - 1s 318ms/step - loss: 1.6652 - accuracy: 0.9773 - val_loss: 2.3672 - val_accuracy: 0.3636
Epoch 7/30
3/3 [==============================] - 1s 315ms/step - loss: 1.1560 - accuracy: 0.9659 - val_loss: 3.6509 - val_accuracy: 0.2727


In [22]:
model.evaluate(X_val,Y_val)

1/1 [==============================] - 0s 66ms/step - loss: 0.2492 - accuracy: 0.9091


[0.24915075302124023, 0.9090909361839294]

In [22]:
model = build_xception_model()
epochs = 30
batch_size = 32
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_29 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense_10 (Dense)            (None, 256)               524544    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 2)                

In [23]:
hist = model.fit(
    X_train,
    Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, Y_val),
    callbacks = [early_stopping])
hist

Epoch 1/30


ValueError: in user code:

    File "/home/alvantor/opt/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/alvantor/opt/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/alvantor/opt/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/alvantor/opt/anaconda3/lib/python3.11/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/alvantor/opt/anaconda3/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/alvantor/opt/anaconda3/lib/python3.11/site-packages/keras/engine/input_spec.py", l